In [1]:
import torch
import json
import numpy as np
import pandas as pd
import nltk
import random
import itertools
import collections
import datasets
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

2024-09-26 17:46:50.547800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 17:46:50.563105: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 17:46:50.567763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 17:46:50.579370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 17:46:51.374829: W tensorflow/compiler/tf2

In [2]:
device = torch.device("mps")

In [3]:
model_name = "microsoft/mdeberta-v3-base"


In [4]:
model_name = "microsoft/mdeberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
tokenizer.model_max_length = 512

/home/arnesh/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/arnesh/.local/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
train = datasets.load_dataset("Isotonic/pii-masking-200k")
train

DatasetDict({
    train: Dataset({
        features: ['masked_text', 'unmasked_text', 'privacy_mask', 'span_labels', 'bio_labels', 'tokenised_text', 'language'],
        num_rows: 209261
    })
})

In [6]:
df = train['train'].to_pandas()

In [7]:
df

,masked_text,unmasked_text,privacy_mask,span_labels,bio_labels,tokenised_text,language
0,The literacy program in [CITY_1] is starting s...,The literacy program in Cliftonfurt is startin...,"{'[CITY_1]': 'Cliftonfurt', '[FIRSTNAME_1]': '...","[[0, 24, 'O'], [24, 35, 'CITY_1'], [35, 54, 'O...","[O, O, O, O, B-CITY, I-CITY, O, O, O, O, B-FIR...","[the, literacy, program, in, clifton, ##furt, ...",en
1,Nous avons identifié les dangers potentiels da...,Nous avons identifié les dangers potentiels da...,"{'[ZIPCODE_1]': '75742', '[DATE_1]': '18th Apr...","[[0, 57, 'O'], [57, 62, 'ZIPCODE_1'], [62, 145...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[nous, avon, ##s, ide, ##nti, ##fie, les, dang...",fr
2,Le spese per l'intervento chirurgico possono e...,Le spese per l'intervento chirurgico possono e...,"{'[CREDITCARDISSUER_1]': 'jcb', '[CREDITCARDNU...","[[0, 144, 'O'], [144, 147, 'CREDITCARDISSUER_1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[le, sp, ##ese, per, l, ', intervento, chi, ##...",it
3,"Hi [USERNAME_1], thank you for subscribing to ...","Hi Eunice_Conroy, thank you for subscribing to...","{'[USERNAME_1]': 'Eunice_Conroy', '[JOBTYPE_1]...","[[0, 3, 'O'], [3, 16, 'USERNAME_1'], [16, 151,...","[O, B-USERNAME, I-USERNAME, I-USERNAME, I-USER...","[hi, eun, ##ice, _, con, ##roy, ,, thank, you,...",en
4,"[PREFIX_1] [LASTNAME_1], we have received a jo...","Mrs. Waters, we have received a job applicatio...","{'[PREFIX_1]': 'Mrs.', '[LASTNAME_1]': 'Waters...","[[0, 4, 'PREFIX_1'], [4, 5, 'O'], [5, 11, 'LAS...","[B-PREFIX, I-PREFIX, B-LASTNAME, O, O, O, O, O...","[mrs, ., waters, ,, we, have, received, a, job...",en
...,...,...,...,...,...,...,...
209256,"For our international day, parents are invited...","For our international day, parents are invited...","{'[IBAN_1]': 'RS39135300042412014026', '[CURRE...","[[0, 90, 'O'], [90, 112, 'IBAN_1'], [112, 130,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[for, our, international, day, ,, parents, are...",en
209257,"Chers citoyens, notre enquête annuelle sur la ...","Chers citoyens, notre enquête annuelle sur la ...",{'[USERAGENT_1]': 'Mozilla/5.0 (Windows NT 5.0...,"[[0, 95, 'O'], [95, 160, 'USERAGENT_1'], [160,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[cher, ##s, citoyens, ,, notre, enquete, annue...",fr
209258,Attività a rischio anomalo segnalate da [IPV6_...,Attività a rischio anomalo segnalate da dbf7:f...,{'[IPV6_1]': 'dbf7:fbf8:4216:e6bf:f8b9:6cae:af...,"[[0, 40, 'O'], [40, 79, 'IPV6_1'], [79, 91, 'O...","[O, O, O, O, O, O, O, O, O, B-IPV6, I-IPV6, I-...","[attivita, a, rischio, ano, ##mal, ##o, segna,...",it
209259,Pour augmenter la diffusion de l'éducation hol...,Pour augmenter la diffusion de l'éducation hol...,"{'[PHONEIMEI_1]': '97-574746-190716-1', '[BUIL...","[[0, 90, 'O'], [90, 108, 'PHONEIMEI_1'], [108,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[pour, augmenter, la, diffusion, de, l, ', edu...",fr


In [8]:
#convert 

In [9]:
kp = 92

In [10]:
# remove more than 512
import IPython
tt = df.tokenised_text.tolist()
tt_lens = [len(tokenizer(list(t), is_split_into_words=True, truncation=True, max_length=512)['input_ids']) for t in tqdm(tt)]
df[['tt_lens']] = 0
df.tt_lens = tt_lens
df = df.loc[df.tt_lens <= 510].reset_index(drop=True)

  0%|          | 0/209261 [00:00<?, ?it/s]

In [11]:
nt = df.bio_labels.tolist()
nt = list(itertools.chain.from_iterable(nt)) # merge the list of lists into one list
nt = collections.Counter(nt) # Get count of each tag
all_labels = list(nt.keys()) # get all unique tags(labels)

In [12]:
source_texts = df.unmasked_text.tolist()
target_texts = df.masked_text.tolist()
tokenized_texts = df.tokenised_text.tolist()
ner_tags = df.bio_labels.tolist()

In [13]:
len(all_labels)

112

In [14]:
source_texts[0]

'The literacy program in Cliftonfurt is starting soon. Reggie, can we count on your presence?'

In [15]:
target_texts[0]

'The literacy program in [CITY_1] is starting soon. [FIRSTNAME_1], can we count on your presence?'

In [16]:
# ## checking if the tokens align
# i = random.randint(0, len(source_texts))
# x0 = tokenizer.convert_ids_to_tokens(tokenizer(source_texts[i])['input_ids'])
# x0.pop(0)  # CLS is not present in the dataset
# for t in zip(x0, tokenized_texts[i]):
#     print(t)

In [17]:
# Create label dict
label2id = dict([(value,key) for key, value in enumerate(all_labels)])
id2label = dict(map(reversed, label2id.items()))

label2id, id2label

({'O': 0,
  'B-CITY': 1,
  'I-CITY': 2,
  'B-FIRSTNAME': 3,
  'I-FIRSTNAME': 4,
  'B-ZIPCODE': 5,
  'I-ZIPCODE': 6,
  'B-DATE': 7,
  'I-DATE': 8,
  'B-CREDITCARDISSUER': 9,
  'I-CREDITCARDISSUER': 10,
  'B-CREDITCARDNUMBER': 11,
  'I-CREDITCARDNUMBER': 12,
  'B-CREDITCARDCVV': 13,
  'B-USERNAME': 14,
  'I-USERNAME': 15,
  'B-JOBTYPE': 16,
  'B-PREFIX': 17,
  'I-PREFIX': 18,
  'B-LASTNAME': 19,
  'B-EMAIL': 20,
  'I-EMAIL': 21,
  'B-IP': 22,
  'I-IP': 23,
  'B-PHONEIMEI': 24,
  'I-PHONEIMEI': 25,
  'B-MAC': 26,
  'I-MAC': 27,
  'B-DOB': 28,
  'I-DOB': 29,
  'B-NEARBYGPSCOORDINATE': 30,
  'I-NEARBYGPSCOORDINATE': 31,
  'B-BUILDINGNUMBER': 32,
  'I-BUILDINGNUMBER': 33,
  'B-IPV4': 34,
  'I-IPV4': 35,
  'B-BITCOINADDRESS': 36,
  'I-BITCOINADDRESS': 37,
  'B-SSN': 38,
  'I-SSN': 39,
  'B-PHONENUMBER': 40,
  'I-PHONENUMBER': 41,
  'B-SEX': 42,
  'B-SECONDARYADDRESS': 43,
  'I-SECONDARYADDRESS': 44,
  'B-ACCOUNTNUMBER': 45,
  'I-ACCOUNTNUMBER': 46,
  'B-ACCOUNTNAME': 47,
  'I-ACCOUNTNAME': 48

In [18]:
for j in tqdm(range(len(ner_tags))):
    tags = ner_tags[j]
    for i in range(len(tags)):
        for k,v in label2id.items():
            if tags[i] == k:
                tags[i] = v
    ner_tags[j] = tags
df.ner_tags = ner_tags

  0%|          | 0/209230 [00:00<?, ?it/s]

/tmp/ipykernel_293641/1087438728.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.ner_tags = ner_tags


In [19]:
ner_tags = [list(ner) for ner in ner_tags]
ner_tags[0]

[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
df[['source_words']] = "source_words"
source_words = [text.split(" ") for text in source_texts]
df.source_words = source_words

In [21]:
# removing rows where the len(tokenized_texts[i]) does not match len(ner_tags[i])
idx = [i for i in range(len(ner_tags)) if len(tokenized_texts[i]) != len(ner_tags[i])]
df = df.drop(index=idx).reset_index(drop=True)

In [22]:
dataset = datasets.Dataset.from_pandas(df)
dataset

Dataset({
    features: ['masked_text', 'unmasked_text', 'privacy_mask', 'span_labels', 'bio_labels', 'tokenised_text', 'language', 'tt_lens', 'source_words'],
    num_rows: 209230
})

In [23]:
dataset[0]

{'masked_text': 'The literacy program in [CITY_1] is starting soon. [FIRSTNAME_1], can we count on your presence?',
 'unmasked_text': 'The literacy program in Cliftonfurt is starting soon. Reggie, can we count on your presence?',
 'privacy_mask': "{'[CITY_1]': 'Cliftonfurt', '[FIRSTNAME_1]': 'Reggie'}",
 'span_labels': "[[0, 24, 'O'], [24, 35, 'CITY_1'], [35, 54, 'O'], [54, 60, 'FIRSTNAME_1'], [60, 92, 'O']]",
 'bio_labels': [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokenised_text': ['the',
  'literacy',
  'program',
  'in',
  'clifton',
  '##furt',
  'is',
  'starting',
  'soon',
  '.',
  'reg',
  '##gie',
  ',',
  'can',
  'we',
  'count',
  'on',
  'your',
  'presence',
  '?'],
 'language': 'en',
 'tt_lens': 32,
 'source_words': ['The',
  'literacy',
  'program',
  'in',
  'Cliftonfurt',
  'is',
  'starting',
  'soon.',
  'Reggie,',
  'can',
  'we',
  'count',
  'on',
  'your',
  'presence?']}

In [24]:
def align_labels(example):
    tokenized_input = tokenizer(example["tokenised_text"], is_split_into_words=True)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    word_ids = tokenized_input.word_ids()
    aligned_labels = [-100 if i is None else example["bio_labels"][i] for i in word_ids]
    tokenized_input['labels'] = aligned_labels
    return tokenized_input

In [25]:
al = align_labels(dataset[0])
print(len(al['input_ids']), len(al['attention_mask']), len(al['labels']))

32 32 32


In [26]:
tokenizer("hello")

{'input_ids': [1, 124394, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [27]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer.encode_plus(examples["tokenised_text"], is_split_into_words=True, truncation=True, max_length=512)

    labels = []
    for i, label in enumerate(examples["bio_labels"]):
        word_ids = tokenized_inputs.word_ids(i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [28]:
x = dataset.map(align_labels, num_proc=8, remove_columns=dataset.column_names)

Map (num_proc=8):   0%|          | 0/209230 [00:00<?, ? examples/s]

In [29]:
tokenized_dataset = x.train_test_split(test_size=0.05)

In [30]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 198768
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10462
    })
})

In [31]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [32]:
import evaluate

metric  = evaluate.load("seqeval")

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=1)
    return pred_ids, labels


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [33]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(all_labels), label2id=label2id, id2label=id2label)
print(model.config)

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2Config {
  "_name_or_path": "microsoft/mdeberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-CITY",
    "2": "I-CITY",
    "3": "B-FIRSTNAME",
    "4": "I-FIRSTNAME",
    "5": "B-ZIPCODE",
    "6": "I-ZIPCODE",
    "7": "B-DATE",
    "8": "I-DATE",
    "9": "B-CREDITCARDISSUER",
    "10": "I-CREDITCARDISSUER",
    "11": "B-CREDITCARDNUMBER",
    "12": "I-CREDITCARDNUMBER",
    "13": "B-CREDITCARDCVV",
    "14": "B-USERNAME",
    "15": "I-USERNAME",
    "16": "B-JOBTYPE",
    "17": "B-PREFIX",
    "18": "I-PREFIX",
    "19": "B-LASTNAME",
    "20": "B-EMAIL",
    "21": "I-EMAIL",
    "22": "B-IP",
    "23": "I-IP",
    "24": "B-PHONEIMEI",
    "25": "I-PHONEIMEI",
    "26": "B-MAC",
    "27": "I-MAC",
    "28": "B-DOB",
    "29": "I-DOB",
    "30": "B-NEARBYGPSCOORDINATE",
    "31": "I-NEARBYGPSCOORDINATE",
    "32": "B-BUILDINGNUMBER",
    "33": "I-BUILDING

In [34]:
training_arguments = TrainingArguments(
    output_dir="output_mdeberta_base",
    max_steps=400000,
    eval_steps=10000,   
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    overwrite_output_dir=True,
    warmup_ratio=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="steps",
    eval_strategy="steps",
    load_best_model_at_end=True,
    save_total_limit=1,
    save_steps=10000,
    lr_scheduler_type="cosine",
    warmup_steps=3000,
    metric_for_best_model="f1",
    greater_is_better=True,
    torch_compile=False,
    eval_accumulation_steps=16,

)
from functools import partial

trainer = Trainer(
    model,
    training_arguments,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=DataCollatorForTokenClassification(tokenizer),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)


max_steps is given, it will override any value given in num_train_epochs


In [35]:
trainer.train()



  0%|          | 0/400000 [00:00<?, ?it/s]

{'loss': 3.1839, 'grad_norm': 7.745646953582764, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.01}
{'loss': 1.2624, 'grad_norm': 6.828775882720947, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.02}
{'loss': 0.7649, 'grad_norm': 4.576783180236816, 'learning_rate': 1e-05, 'epoch': 0.03}
{'loss': 0.5121, 'grad_norm': 3.3890411853790283, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.04}
{'loss': 0.3571, 'grad_norm': 2.940311908721924, 'learning_rate': 1.6666666666666667e-05, 'epoch': 0.05}
{'loss': 0.2738, 'grad_norm': 3.4379637241363525, 'learning_rate': 2e-05, 'epoch': 0.06}
{'loss': 0.2062, 'grad_norm': 4.231479644775391, 'learning_rate': 1.999992172408044e-05, 'epoch': 0.07}
{'loss': 0.1856, 'grad_norm': 8.920188903808594, 'learning_rate': 1.999968689754717e-05, 'epoch': 0.08}
{'loss': 0.1577, 'grad_norm': 1.1247848272323608, 'learning_rate': 1.9999295524076455e-05, 'epoch': 0.09}
{'loss': 0.1524, 'grad_norm': 1.2960718870162964, 'learning_rate': 1.999874760979531e-05,

  0%|          | 0/5231 [00:00<?, ?it/s]

/home/arnesh/.local/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.09239111095666885, 'eval_precision': 0.9134922323898702, 'eval_recall': 0.9294746871075311, 'eval_f1': 0.9214141587601425, 'eval_accuracy': 0.9684061856813415, 'eval_runtime': 116.0891, 'eval_samples_per_second': 90.12, 'eval_steps_per_second': 45.06, 'epoch': 0.2}
{'loss': 0.0829, 'grad_norm': 1.6202961206436157, 'learning_rate': 1.9982393064285106e-05, 'epoch': 0.21}
{'loss': 0.0953, 'grad_norm': 0.23380757868289948, 'learning_rate': 1.997996803002123e-05, 'epoch': 0.22}
{'loss': 0.0923, 'grad_norm': 21.746740341186523, 'learning_rate': 1.99773867575224e-05, 'epoch': 0.23}
{'loss': 0.0869, 'grad_norm': 0.755976676940918, 'learning_rate': 1.997464928719892e-05, 'epoch': 0.24}
{'loss': 0.09, 'grad_norm': 0.14635367691516876, 'learning_rate': 1.9971755661906376e-05, 'epoch': 0.25}
{'loss': 0.0843, 'grad_norm': 0.42474454641342163, 'learning_rate': 1.9968705926945015e-05, 'epoch': 0.26}
{'loss': 0.0927, 'grad_norm': 11.507859230041504, 'learning_rate': 1.9965500130058996e

...

{'eval_loss': 0.04117940366268158, 'eval_precision': 0.9841746189386416, 'eval_recall': 0.9870728855398208, 'eval_f1': 0.9856216216216217, 'eval_accuracy': 0.9931763101843244, 'eval_runtime': 116.5383, 'eval_samples_per_second': 89.773, 'eval_steps_per_second': 44.887, 'epoch': 6.04}
{'loss': 0.0059, 'grad_norm': 0.0005338359624147415, 'learning_rate': 2.942934942509171e-06, 'epoch': 6.05}
{'loss': 0.0023, 'grad_norm': 0.00019953699666075408, 'learning_rate': 2.914957108503722e-06, 'epoch': 6.06}
{'loss': 0.0011, 'grad_norm': 0.0003969695244450122, 'learning_rate': 2.8870901921477656e-06, 'epoch': 6.07}
{'loss': 0.0042, 'grad_norm': 136.90921020507812, 'learning_rate': 2.8593346297030077e-06, 'epoch': 6.08}
{'loss': 0.0027, 'grad_norm': 3.0196447372436523, 'learning_rate': 2.831690855687882e-06, 'epoch': 6.09}
{'loss': 0.0029, 'grad_norm': 0.0009481729939579964, 'learning_rate': 2.8041593028707513e-06, 'epoch': 6.1}
{'loss': 0.0033, 'grad_norm': 0.0006036604172550142, 'learning_rate': 

  0%|          | 0/5231 [00:00<?, ?it/s]

{'eval_loss': 0.044598232954740524, 'eval_precision': 0.9849103037368047, 'eval_recall': 0.9879173704040535, 'eval_f1': 0.9864115453218746, 'eval_accuracy': 0.9933959381866773, 'eval_runtime': 115.8188, 'eval_samples_per_second': 90.331, 'eval_steps_per_second': 45.165, 'epoch': 6.24}
{'loss': 0.0006, 'grad_norm': 0.00043678979272954166, 'learning_rate': 2.4049414036242057e-06, 'epoch': 6.25}
{'loss': 0.0017, 'grad_norm': 0.3724506199359894, 'learning_rate': 2.379262792479504e-06, 'epoch': 6.26}
{'loss': 0.0073, 'grad_norm': 0.00048407024587504566, 'learning_rate': 2.3537034853773357e-06, 'epoch': 6.27}
{'loss': 0.0035, 'grad_norm': 0.005040855146944523, 'learning_rate': 2.328263882453353e-06, 'epoch': 6.28}
{'loss': 0.0011, 'grad_norm': 0.0027064448222517967, 'learning_rate': 2.3029443819692255e-06, 'epoch': 6.29}
{'loss': 0.007, 'grad_norm': 0.0030472457874566317, 'learning_rate': 2.2777453803063834e-06, 'epoch': 6.3}
{'loss': 0.0011, 'grad_norm': 0.036084357649087906, 'learning_rate

  0%|          | 0/5231 [00:00<?, ?it/s]

{'eval_loss': 0.044236920773983, 'eval_precision': 0.9846937674057124, 'eval_recall': 0.9876575289073665, 'eval_f1': 0.9861734214025492, 'eval_accuracy': 0.9932143645411677, 'eval_runtime': 115.48, 'eval_samples_per_second': 90.596, 'eval_steps_per_second': 45.298, 'epoch': 6.44}
{'loss': 0.0014, 'grad_norm': 0.0037304668221622705, 'learning_rate': 1.9144839284536177e-06, 'epoch': 6.45}
{'loss': 0.0011, 'grad_norm': 0.3985676169395447, 'learning_rate': 1.8912652578362857e-06, 'epoch': 6.46}
{'loss': 0.006, 'grad_norm': 0.0027015123050659895, 'learning_rate': 1.8681735309526438e-06, 'epoch': 6.47}
{'loss': 0.0019, 'grad_norm': 0.0003821970894932747, 'learning_rate': 1.8452091093079217e-06, 'epoch': 6.48}
{'loss': 0.0053, 'grad_norm': 0.7266119122505188, 'learning_rate': 1.822372352414361e-06, 'epoch': 6.49}
{'loss': 0.0022, 'grad_norm': 0.0018061378505080938, 'learning_rate': 1.7996636177855931e-06, 'epoch': 6.5}
{'loss': 0.0021, 'grad_norm': 0.0001268090563826263, 'learning_rate': 1.77

In [ ]:
{'eval_loss': 0.044236920773983, 'eval_precision': 0.9846937674057124, 'eval_recall': 0.9876575289073665, 'eval_f1': 0.9861734214025492, 'eval_accuracy': 0.9932143645411677, 'eval_runtime': 115.48, 'eval_samples_per_second': 90.596, 'eval_steps_per_second': 45.298, 'epoch': 6.44}
